In [ ]:
# Load some libraries for reading the data from the database and plotting 

import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from scoping.tasks import *
from tmv_app.tasks import *

from django.db.models import Count, Sum

pids = [148, 182, 147, 186]

queries = Query.objects.filter(
    text="[GENERATED TYPE 1]",
    project__in=pids
)

all_docs = set([])

for q in queries.order_by('project_id'):
    print(f"\n{q.title}")
    print(q.r_count)
    q.doc_set.clear()
    if q.project.id == 186:
        print("assigning all docs")
        docs = all_docs
    else:
        docs = set(Doc.objects.filter(query__project=q.project).values_list('pk',flat=True))
        all_docs = all_docs | docs
    Through = Doc.query.through
    dqs = [Through(doc_id=d,query=q) for d in docs]
    Through.objects.bulk_create(dqs)
    q.r_count = q.doc_set.count()
    q.save()
    print(q.r_count)

queries


In [ ]:
RunStats.objects.filter(query__in=queries).delete()

In [ ]:

for method in ["NM","LD"]:
    for K in [20,40,60,80, 100, 120, 140 ]:
        for q in queries.order_by('r_count'):
            for alpha in [0.05,0.1]:
                if method=="LD":
                    alpha = alpha*10  
                model, created = RunStats.objects.get_or_create(
                    method=method,
                    min_freq=10,
                    fancy_tokenization=True,
                    K=K,
                    alpha=alpha,
                    query=q,
                )
                if created or model.status < 3:
                    do_nmf(model.pk)